In [4]:
!pip install scikit-learn



   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.1 MB 1.7 MB/s eta 0:00:07
    --------------------------------------- 0.2/11.1 MB 2.0 MB/s eta 0:00:06
   - -------------------------------------- 0.3/11.1 MB 2.2 MB/s eta 0:00:05
   - -------------------------------------- 0.5/11.1 MB 2.6 MB/s eta 0:00:05
   -- ------------------------------------- 0.6/11.1 MB 2.5 MB/s eta 0:00:05
   -- ------------------------------------- 0.7/11.1 MB 2.6 MB/s eta 0:00:05
   --- ------------------------------------ 0.9/11.1 MB 2.8 MB/s eta 0:00:04
   --- ------------------------------------ 1.0/11.1 MB 2.8 MB/s eta 0:00:04
   ---- ----------------------------------- 1.2/11.1 MB 2.8 MB/s eta 0:00:04
   ---- ----------------------------------- 1.4/11.1 MB 3.0 MB/s eta 0:00:04
   ----- ---------------------------------- 1.5/11.1 MB 2.9 MB/s eta 0:00:04
   ----- ----


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import csv
file_path = "C:/project/news_cleaned_2018_02_13.csv"
csv.field_size_limit(100000000)


selected_columns = ["id", "domain", "type", "url", "content", "title"]
sample_chunks = []
for chunk in pd.read_csv(file_path, usecols=selected_columns, chunksize=100000, on_bad_lines='warn', engine='python'):
    sample = chunk.sample(frac=0.1, random_state=42)
    sample_chunks.append(sample)
    df = pd.concat(sample_chunks, ignore_index=True)

train_dataframe, test_dataframe = train_test_split(df, test_size=0.2, random_state=42)


train_dataframe.to_csv("C:/project/train_data.csv", chunksize=100000)
test_dataframe.to_csv("C:/project/test_data.csv", chunksize=100000)


In [ ]:
import re
import string
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from collections import Counter
import csv
import ast
import contractions
import matplotlib.pyplot as plt

csv.field_size_limit(100000000)

nltk.download('stopwords')

def tokenizer(text):
    # contractions will remove any additional english contractions such as "'s", "n't"
    # this only works in english chars, verifying if there is non english chars
    if text.isascii():
        text = contractions.fix(text)
    tokens = nltk.word_tokenize(text)
    return tokens

# features to check from data
url_pattern = re.compile(r'https?://\S+|www\.\S+')
# should match formats such as "%m %d,%Y", "%m %d" and numeric dates in format YYYY-MM-DD, DD/MM/YYYY, MM-DD-YYY
date_pattern = re.compile(r'\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \d{1,2}(?:, \d{4})?\b | \b\d{1,2}[-/.]\d{1,2}[-/.]\d{2,4}\b | \b\d{4}[-/.]\d{1,2}[-/.]\d{1,2}\b')
# should match decimals and integers
numbers_pattern = re.compile(r'\b\d+(\.\d+)?\b')
url_count = 0
date_count = 0
numbers_count = 0

def count_features(text):
    urls = len(url_pattern.findall(text))
    dates = len(date_pattern.findall(text))
    nums = len(numbers_pattern.findall(text))
    return urls, dates, nums


train_data_path = 'C:/project/train_data.csv'
tokens_output_file_path = 'C:/project/train_data_tokens.csv'

chunks = []
with open(tokens_output_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["tokens"])
    for chunk in pd.read_csv(train_data_path , chunksize=10000, on_bad_lines='warn', engine='python'):
        # remove NaN values from training data to apply tokenization
        chunk['content'] = chunk['content'].astype(str).fillna('')
        chunk_tokens = (chunk['content'].apply(tokenizer))
        writer.writerows([tokens] for tokens in chunk_tokens)
        chunks.append(chunk)
        for content in chunk['content']:
            urls, dates, nums = count_features(content)
            url_count += urls
            date_count += dates
            numbers_count += nums

# train_data
train_data  = pd.concat(chunks, ignore_index=True)


# check how divided the data is
print(train_data['type'].value_counts())
# show features in text
print(f"Total URLs: {url_count}")
print(f"Total Dates: {date_count}")
print(f"Total Numbers: {numbers_count}")

# get most common tokens from the tokenized file
stop_words = stopwords.words('english')
punct = [char for char in string.punctuation]
# remove additional punctuation chars
extra_punct = ['’', '“', '”', '—', '–', '‘', '...', '…', '``', "''", '--']

token_counter = Counter()
stop_words_token_counter = Counter()
stemmed_token_counter = Counter()

stemmer = SnowballStemmer('english')

with open(tokens_output_file_path, newline='', encoding='utf-8') as file:
    reader = csv.reader(file)
    next(reader) # skipping header 'token'
    for row in reader:
        # the format of each row is string and needs to be converted to list
        tokens = ast.literal_eval(row[0])
        tokens = [token for token in tokens if token not in punct and token not in extra_punct]
        token_counter.update(tokens)
        # remove stopwords and add them to separate counter
        tokens_without_stopwords = [token for token in tokens if token.lower() not in stop_words]
        stop_words_token_counter.update(tokens_without_stopwords)
        # apply stemming
        stemmed_token_counter.update(stemmer.stem(token) for token in tokens_without_stopwords)

most_common_tokens = token_counter.most_common(100)
print(f"100 most common tokens: {most_common_tokens}")
most_common_tokens_without_stopwords = stop_words_token_counter.most_common(100)
print(f"100 most common tokens without stopwords: {most_common_tokens_without_stopwords}")
most_common_tokens_after_stemming = stemmed_token_counter.most_common(100)
print(f"100 most common tokens after stemming: {most_common_tokens_after_stemming}")

original_vocabulary_size = token_counter.total()
stop_words_vocabulary_size = stop_words_token_counter.total()
stemmed_vocabulary_size = stemmed_token_counter.total()

# Compute reduction rates
stopwords_reduction_rate = ((original_vocabulary_size- stop_words_vocabulary_size) / original_vocabulary_size) * 100
stemming_reduction_rate = ((stop_words_vocabulary_size - stemmed_vocabulary_size) / stop_words_vocabulary_size) * 100
print(f"Original vocabulary size: {original_vocabulary_size}")
print(f"Vocabulary size after removing stopwords: {stop_words_vocabulary_size}")
print(f"Reduction rate after removing stopwords: {stopwords_reduction_rate}")
print(f"Vocabulary size after stemming: {stemmed_vocabulary_size}")
print(f"Reduction rate after stemming: {stemming_reduction_rate}")

# visualize most common words
words, counts = zip(*most_common_tokens_without_stopwords)
plt.figure(figsize=(15, 5))
plt.bar(words, counts)
plt.xticks(rotation=90)
plt.xlabel("Words")
plt.ylabel("Frequency")
plt.title("Top 100 Most Frequent Words After Stopwords")
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import csv
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Increase field size for large CSV parsing
csv.field_size_limit(100000000)

# Load train data
train_file_path = "C:/project/train_data.csv"
selected_columns = ["type", "content"]

train_data_chunks = []
for chunk in pd.read_csv(train_file_path, usecols=selected_columns, chunksize=100000, on_bad_lines='warn', engine='python'):
    chunk = chunk.dropna()
    train_data_chunks.append(chunk)
train_data = pd.concat(train_data_chunks, ignore_index=True).sample(n=1000, random_state=42)

# Load test data
test_file_path = "dataset/test_data.csv"
test_data_chunks = []
for chunk in pd.read_csv(test_file_path, usecols=selected_columns, chunksize=100000, on_bad_lines='warn', engine='python'):
    chunk = chunk.dropna()
    test_data_chunks.append(chunk)
test_data = pd.concat(test_data_chunks, ignore_index=True).sample(n=1000, random_state=42)

# Binary classification: fake (0) vs not fake (1)
fake_labels = {'fake', 'satire', 'bias', 'conspiracy', 'state', 'junksci', 'hate', 'clickbait', 'unreliable'}
not_fake_labels = {'political', 'reliable'}

train_data['binary_label'] = train_data['type'].apply(lambda x: 1 if x in not_fake_labels else 0)
test_data['binary_label'] = test_data['type'].apply(lambda x: 1 if x in not_fake_labels else 0)

# Features and labels
X_train = train_data['content']
y_train = train_data['binary_label']
X_test = test_data['content']
y_test = test_data['binary_label']

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Naive Bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train_vec, y_train)
y_pred = nb_model.predict(X_test_vec)

# Evaluation
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary')
recall = recall_score(y_test, y_pred, average='binary')
f1 = f1_score(y_test, y_pred, average='binary')

print("Naive Bayes Classification Results:")
print(f"Accuracy  : {accuracy:.4f}")
print(f"Precision : {precision:.4f}")
print(f"Recall    : {recall:.4f}")
print(f"F1 Score  : {f1:.4f}")
